# Recurrent neural networks

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

Trains a memory network on the bAbI dataset.

References:
- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698

- Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895

Reaches 98.6% accuracy on task 'single_supporting_fact_10k' after 120 epochs.
Time per epoch: 3s on CPU (core i7).

In [ ]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

Using TensorFlow backend.


524288/600901 [=========================>....] - ETA: 0scorpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 674s - loss: 2.2437   

----- diversity: 0.2
----- Generating with seed: "relation
(grundstellung) to all things.
"
relation
(grundstellung) to all things.










=lostincessions and the wisters and and the ways and and and the sore and and which and the and and in are the supporenters and in the surion of the reasion of the stirale the surilitions and and which and and the supporentens of the reared the proporenter to the stireding and which and the wisters and the sore and the sore the streat of a the man to the supporitions and and in the reara
----- diversity: 0.5
----- Generating with seed: "relation
(grundstellung) to all things.
"
relation
(grundstellung) to all things.









                                

/home/fabian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log


l period, and and couraded in the strange of the constant to the new strain ter, is the way
----- diversity: 1.0
----- Generating with seed: "her all mechanical action, inasmuch as a"
her all mechanical action, inasmuch as a rightinal offerthen in a strengiasive, is theirs, in any lifice at med to very yecition ancell
in what is surcre
nok
and any thinks. it is a spiriity
my been nothing moralared, is the which, emphomishs, and really the privisely inflict to
fact reward to mistnes,
whole is no belief good; now regutatere ill for pling la with
"amming
the uncemmitates with has founding besperiding
pussed.--wi gladict
----- diversity: 1.2
----- Generating with seed: "her all mechanical action, inasmuch as a"
her all mechanical action, inasmuch as ascenolae
finties, by playe
loveds klet and philosophes
having can bling of
whole
cas as on may dramestly: e
brenoldo, hind fried
yegerait all rsed.
s appard, no every bad, ow to
innorenal and but garm timeiative in orrual polig itsive. in eari

ncipal moral judgments, europe has become aieseared the aies aieioesion ooioeationsiesineas aiestion oioesiaily stiling of aimality aies a stranofice and ane and a ioldered aim of the stioed the personationaieaing aim ofieit to a sioeaieary of the and andeart aies and the areadeared aies and sueger toal and aies aon a iomineal of theioious and and aies the senealed the areineding and the aieseaingedinged aies and and and theaing the sue
----- diversity: 0.5
----- Generating with seed: "ncipal moral judgments, europe has becom"
ncipal moral judgments, europe has become ioeveaned analeeitsoeaieiseationaieatian
oimer of relieain oimsinged and same dactioeed ioeat,inificeal sieg the maies. heareed ane and aneear theeiseaisedied wiedioes, aid, and oodedeiailyinatedion aiewilieitain, the woritionians ofing andainiteait,odedeainineas ofealed the live the adeeationity anared aim andiois,oratiois ioive. to amongain woed, aiditedityaliedieitive thearkines anelameitedit
----- diversity: 1.0
-----

nvironment, in ourselves, not in the mureaieinaieioioioioeievioioioeeaieaieaieeieaieioeeeeieeeaieaieioioioeeieaieeeeeaieeeaieeaieieeieaieioioaieioioeieaieeeioiaieaieeeieieeaieaieaieeaieioieioioioioioeeaieieaieioiaieiaieaieeaieioeaieioioiaieioeeaieeaeieaieaieaieeeeeeaieeoieieieaieieaieeeieieieeaieeaieioaieeaieaieooioioioioeaieeaieieeaoioeeaeieeeioaieaieioioioeeaieeoioeeieaieioioeieaieaieaaieioeeooooeeaieaieeeeaieeeeeeeoooeaieaoioaieaieio
----- diversity: 1.2
----- Generating with seed: "nvironment, in ourselves, not in the mur"
nvironment, in ourselves, not in the mureeaie aieioioiieaieitaieioaieeiieeaieaieaieeeeaeieaieeeaieiaieaieeeieaieeeaieioieioaieioooeaeaieeaieieeaieeaeieaieaoieaieeaeieeaeoieieeieaieaieeeaieaoeeieioiaeieeieaieioaieioioieaieeaieeieooeeeeeeaieeaieiaieeaiieaieieaieioieioaieaieioieaieaieeaieooeaieeeeaeieaieaieioeeaieioioiooieieieaeieaaieaieeaieieeaioieaaieieieeeaoeoeeaieeeieeaeioeaieiaieaieaieeeaooooooooeaeieeeeaieeieaieeaieioaieioieeaieiaiei
--------------------------

onality in the general constitution of more the come_itice the string and vain the_ the string an_ te_ the comes the someits ies the string of ties teem teem the survigitiäc, the s_ the_ the strianole_--the som varieient tho_ollion to the ents oed varient the_ ten_" the seculeity and caster this iom the streind the stronger "stigator the seeed virtues oes the éblom onotëity aiused the sur_ed and iä--a flection, what_ is te_ to the comes
----- diversity: 0.5
----- Generating with seed: "onality in the general constitution of m"
onality in the general constitution of man and virtue_ is the liber us the
trat woh its preising foe_ feornees fious qeeæ-about and fatæ. with the a selvion like teems for "so comesed
oent the feeline_ respicy and virtue agains_ede_ lange in orieses of his e_ëk all fromen but one woul ughe_ tay vaniäg, lange and ught ases_goowee, the an at_ to te_ presio_erronseindieice oo "fully to tæventy, which the strugh_ virtue moral the surve_ed a
----- diversity: 1.0
-----

pressure and hammer of which a conscienoaiaeeaiaeiieeeiaaoeioieeeaeaieiooiaeiaeoiaieaiooioeieeieeaaieaaaaiieeieeaeaeeieeoeeoieioeeaioeeaaeeaieeeieaeeieaieaeooaiiaiieieeeeiaeaeaeeaeeieeieeiaeeeeeaaeeiaeeiaioieaeeiaeaoaeaoaeieeeaieaioieoaeieaaeaiiieaeieieaioiooeeeiieiaiaieeaaeaeiaeeeiioiieoieiieaoeeoaeeeooaeeeaaieaieaooeeaeeieeeoieieeiaiaeoeaoieieieeeeieeieeeeaaoiaiaeeaieeeiaiaaeiieaaieeeeieeeiaeaieeeeeieeeaaiaaeieiaeiooooeaaeeeieaiaeeie
----- diversity: 1.2
----- Generating with seed: "
pressure and hammer of which a conscien"

pressure and hammer of which a conscieniaieiooaaeeeeeeeeeiaiaeeeaoaiaieooiiaaeaaiaaeaeeieeeaeeaoaeiieeiaeaaiaieaeieaoaieioeeeaaoeeeeeoaiaiaeeaaeeaaieieieoeaoeeeeeeaeeoieeeiooioiaieaeieeaieeaeeieaeeioeaoeeeeiieaieeeeaoeeioaeaiiieaeieeaiooeaieiaeieoaiiiieaiaaoaiiaoeiiieaieoeeoiieeeeieaeeeoaioiaieooeeaaoeoeaaeieaieeooeieaeioeooaeeiaeaeeaieeeieeeiiaieoeoeiieieaioeeaeaeeaoeeaeaeieiaoeeaeeiooeiooeeaieaioieaioaieaaioeaieieeeieiiieiiaeaieaieai
---------------------------

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re


Using TensorFlow backend.


In [3]:

def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.

    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format

    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.

    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])


Extracting stories for the challenge: single_supporting_fact_10k


/home/fabian/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')


In [4]:

print('-')
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)



-
Vectorizing the word sequences...


In [9]:
inputs_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  5, 16, 19, 18,  9,  1,  4, 21, 19, 18, 12,  1], dtype=int32)

In [8]:

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))


-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-
Compiling...
Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 1s - loss: 1.9416 - acc: 0.1735 - val_loss: 1.7920 - val_acc: 0.1530
Epoch 2/120
10000/10000 [==============================] - 1s - loss: 1.7384 - acc: 0.2493 - val_loss: 1.5770 - val_acc: 0.3670
Epoch 3/120
10000/10000 [==============================] - 1s - loss: 1.5583 - acc: 0.3696 - val_loss: 1.4858 - val_acc: 0.4240
Epoch 4/120
10000/10000 [==============================] - 1s - loss: 1.5186 - acc: 0.3953 - val_loss: 1.4692 - val_acc: 0.4220
Epoch 5/120
10000/10000 [==============================] - 1s - 

10000/10000 [==============================] - 2s - loss: 0.4851 - acc: 0.8168 - val_loss: 0.5289 - val_acc: 0.7990
Epoch 62/120
10000/10000 [==============================] - 2s - loss: 0.4583 - acc: 0.8268 - val_loss: 0.4725 - val_acc: 0.8140
Epoch 63/120
10000/10000 [==============================] - 2s - loss: 0.4363 - acc: 0.8385 - val_loss: 0.4469 - val_acc: 0.8330
Epoch 64/120
10000/10000 [==============================] - 1s - loss: 0.4082 - acc: 0.8485 - val_loss: 0.4380 - val_acc: 0.8230
Epoch 65/120
10000/10000 [==============================] - 1s - loss: 0.3935 - acc: 0.8535 - val_loss: 0.4354 - val_acc: 0.8370
Epoch 66/120
10000/10000 [==============================] - 2s - loss: 0.3754 - acc: 0.8610 - val_loss: 0.3848 - val_acc: 0.8560
Epoch 67/120
10000/10000 [==============================] - 2s - loss: 0.3622 - acc: 0.8634 - val_loss: 0.3906 - val_acc: 0.8510
Epoch 68/120
10000/10000 [==============================] - 2s - loss: 0.3513 - acc: 0.8661 - val_loss: 0.3720

NameError: name 'models' is not defined